### Plan B:
ECG Impressions + Clinical Features -> LLM -> 10 responses -> Get most common prediction and most representative response

In [1]:
# Import packages
import json
from huggingface_hub import login
from transformers import AutoModel, AutoModelForCausalLM, AutoTokenizer, pipeline
import transformers
import random
import torch
import time
import re
from tqdm import tqdm
import pandas as pd

# Set GPUs
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"

print("Available GPUs:", torch.cuda.device_count())
for i in range(torch.cuda.device_count()):
    print(f"GPU {i}: {torch.cuda.get_device_name(i)}")

# Set seeds
random.seed(0)
torch.manual_seed(0)

/home/sswee/myenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Available GPUs: 2
GPU 0: NVIDIA A100-SXM4-40GB
GPU 1: NVIDIA A100-SXM4-40GB


In [2]:
# Log into Huggingface
with open("../../huggingface_token.txt", "r") as file:
    access_token = file.read().strip()
login(access_token)

# Load Huggingface Model
model_name = "meta-llama/Llama-3.1-8B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, token=access_token, low_cpu_mem_usage=True,
                    torch_dtype=torch.float16, device_map='auto')

Loading checkpoint shards: 100%|██████████████████| 4/4 [00:07<00:00,  1.78s/it]


In [39]:
def get_message(note):
    system = 'You are a cardiologist. Your task is to generate a prognosis for the next few years with reasoning. The three possible prognoses are survivor, sudden cardiac death, and pump failure death. Please structure the results as follows: PREDICTION: [SURVIVOR | SUDDEN CARDIAC DEATH | PUMP FAILURE DEATH] \n REASONING: [Your explanation here]'
    prompt = f"Here is the patient data: \n{note}"

    messages = [
		{"role": "system", "content": system},
		{"role": "user", "content": prompt}
	]

    return messages

def extract_assistant_response(response):
    parts = response.split("assistant\n\n", 1)
    return parts[1].strip() if len(parts) > 1 else response

In [40]:
# Load in csv file with prompts
df = pd.read_csv("../Data/subject-info-cleaned-with-prompts.csv")
# print(df['Prompts'][0])

In [44]:
# Test message
message = get_message(df['Prompts'][0])
message

[{'role': 'system',
  'content': 'You are a cardiologist. Your task is to generate a prognosis for the next few years with reasoning. The three possible prognoses are survivor, sudden cardiac death, and pump failure death. Please structure the results as follows: PREDICTION: [SURVIVOR | SUDDEN CARDIAC DEATH | PUMP FAILURE DEATH] \n REASONING: [Your explanation here]'},
 {'role': 'user',
  'content': 'Here is the patient data: \nAge: 58\nGender: Male \nWeight: 83 kg\nHeight: 163 cm\nNYHA Class: III\nBlood Pressure: 110/75 mmHg\nPast Medical History: Idiopathic dilated cardiomyopathy\nAlbumin (g/L): 42,4\nALT or GPT (IU/L): 10.0\nAST or GOT (IU/L): 20.0\nTotal Cholesterol (mmol/L): 5,4\nCreatinine (mmol/L): 106.0\nGamma-glutamil transpeptidase (IU/L): 20\nGlucose (mmol/L): 5,7\nHemoglobin (g/L): 132.0\nHDL (mmol/L): 1,29\nPotassium (mEq/L): 4,6\nLDL (mmol/L): 3,36\nSodium (mEq/L): 141.0\nPro-BNP (ng/L): 1834.0\nProtein (g/L): 69\nT3 (pg/dL): 0,05\nT4 (ng/L): 15\nTroponin (ng/mL): 0,01\nT

In [45]:
# Put message into LLM
input_text = tokenizer.apply_chat_template(message, tokenize = False, add_generation_prompt = True)
inputs = tokenizer(input_text, return_tensors = "pt").to(model.device)
output = model.generate(**inputs, max_new_tokens = 1000)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [46]:
# Get result
result = tokenizer.decode(output[0], skip_special_tokens = True)
result = result.replace("**", "")
result = extract_assistant_response(result)
print(result)

PREDICTION: SURVIVOR
REASONING: 

This patient has a complex medical history, including idiopathic dilated cardiomyopathy, which is a significant risk factor for heart failure. However, the patient's current medications (Beta Blockers, Digoxin, Loop Diuretics, ACE Inhibitor) are standard treatments for heart failure and are aimed at reducing symptoms and slowing disease progression. 

The patient's LVEF of 35% indicates a moderately reduced ejection fraction, but it is not severely reduced, and the patient is currently tolerating these symptoms with NYHA Class III, indicating some limitations but not severe symptoms.

The patient's laboratory results show elevated Pro-BNP levels, which is a marker of heart failure, but the albumin levels are within a relatively normal range, suggesting that the patient is not severely malnourished. The liver enzymes are also within a relatively normal range, which suggests that the patient's liver function is intact.

The patient's ECG shows polymorphi